In [1]:
import glob
# 긍정리뷰 100개 불러오기
pos_review=(glob.glob("c:/data/imdb/train/pos/*.txt"))[:100]
lines_pos=[]
for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue
len(lines_pos)

100

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')
# TF-IDF 가중치 할당
vec = TfidfVectorizer(stop_words=stop_words)
vector_lines_pos = vec.fit_transform(lines_pos)
print(vector_lines_pos)
A=vector_lines_pos.toarray()
print(A.shape)
print(A)
# x축 단어, y축 문서


  (0, 2632)	0.10430305595720811
  (0, 1348)	0.10430305595720811
  (0, 1318)	0.07242968766333299
  (0, 3580)	0.05164463226548277
  (0, 146)	0.08961436013929301
  (0, 129)	0.09571071971922811
  (0, 2176)	0.04914865560743787
  (0, 1250)	0.07242968766333299
  (0, 3892)	0.09571071971922811
  (0, 2470)	0.03485607054883035
  (0, 3023)	0.10430305595720811
  (0, 1859)	0.10430305595720811
  (0, 2070)	0.081022023901313
  (0, 630)	0.0749256643213779
  (0, 2855)	0.10430305595720811
  (0, 1798)	0.09571071971922811
  (0, 491)	0.10430305595720811
  (0, 3697)	0.09571071971922811
  (0, 2916)	0.10430305595720811
  (0, 3404)	0.20860611191441622
  (0, 1179)	0.10430305595720811
  (0, 3050)	0.06633332808339788
  (0, 1974)	0.08488565627825163
  (0, 3067)	0.10430305595720811
  (0, 2907)	0.08961436013929301
  :	:
  (99, 1726)	0.08226721859378414
  (99, 1226)	0.06764092294931459
  (99, 2374)	0.08896092696102158
  (99, 1315)	0.08537419078875624
  (99, 194)	0.09320313545124304
  (99, 1532)	0.08537419078875624
  (9

In [3]:
# 현재 상태는 100개의 문서의 유사도
# 단어간의 유사도를 구하는 것이 목적이므로
# 단어-문서 행렬로 변경
# x축 문서, y축 단어
A=A.transpose()
print(A.shape)
print(A)

(4001, 100)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.06538462 0.23078109]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [4]:
import numpy as np
from scipy import sparse
# 밀집행렬 (dense array): 0을 포함하여 모든 값이 다 채워진 행렬
a=np.array([[0.5,0,0],[0,1,0],[0.7,0,1.5]])
# (sparse array) 밀집행렬을 희소행렬로 변환
# 밀집행렬의 단점 : 0이 많을 경우 메모리 낭비가 될 수 있음
# 희소행렬은 0이 아닌 값들의 위치와 값만 기록하여 메모리를 절약하는 방식
b=sparse.csr_matrix(a) # 희소행렬: 0이 아닌 값만 채워진 행렬(메모리 낭비x)
print('{}'.format(b))
# (0,0) 0.5 => 인덱스 0,0에 값 0.5
# (1,1) 1 => 인덱스 1,1에 값 1
# (2,0) 0.7 => 인덱스 2,0에 값 0.7
# (2,2) 1 => 인덱스 2,2에 값 1.5
c=b.toarray() # 희소행렬을 밀집행렬로 변환
print(c)


  (0, 0)	0.5
  (1, 1)	1.0
  (2, 0)	0.7
  (2, 2)	1.5
[[0.5 0.  0. ]
 [0.  1.  0. ]
 [0.7 0.  1.5]]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
A_sparse = sparse.csr_matrix(A)
# 코사인 유사도(같은 방향인지 반대방향인지 확인) 계산(A_sparse 필요)
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
# todok() 행렬을 딕셔너리 형태로 변환
list(similarities_sparse.todok().items())[35000:35010]
# list(similarities_sparse.todok().items())[-10:]
#단어 이름이 아닌 인덱스 형태로 출력됨
# (1469,108), 0.37 1469 단어와 108 단어의 유사도는 37%
# (3968, 3988 단어의 코사인 유사도는 1.0

[((1469, 108), 0.37803585968894865),
 ((1470, 108), 0.2189685434746738),
 ((1476, 108), 0.06407477897013734),
 ((1477, 108), 0.185189577514238),
 ((1480, 108), 0.20111036876169444),
 ((1489, 108), 0.06995711757772019),
 ((1496, 108), 0.10714874067068783),
 ((1503, 108), 0.30487333830091773),
 ((1504, 108), 0.30487333830091773),
 ((1512, 108), 0.30487333830091773)]

In [6]:
vec.get_feature_names()[1469]

c:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'fraud'

In [7]:
vec.get_feature_names()[108]
#fraud actual의 유사도는 37%

'actual'

In [8]:
import pandas as pd
import numpy as np
# 위의 결과값을 데이터프레임으로 출력
df=pd.DataFrame(list(similarities_sparse.todok().items()),columns=["words","weight"])
df2=df.sort_values(by=['weight'],ascending=False)
df2=df2.reset_index(drop=True)
# 단어 자신끼리의 짝은 1이 되므로 1보다 작은 항목들만 출력시킴
df3=df2.loc[np.round(df2['weight']) < 1]
df3=df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(3971, 3372)",0.499961
1,"(3372, 3971)",0.499961
2,"(1192, 2554)",0.499958
3,"(2554, 1192)",0.499958
4,"(2468, 1321)",0.499957
5,"(2468, 710)",0.499957
6,"(710, 2468)",0.499957
7,"(1321, 2468)",0.499957
8,"(2146, 889)",0.499909
9,"(889, 2146)",0.499909
